# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [3]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [4]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [5]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5918)

### Get IDF(Inverse Document Frequency)

In [6]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5918,)

### Get TF-IDF Table

In [7]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [8]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        sent_vec += get_word_vec(word) * get_tf_idf(word, i)
    
    return sent_vec / len(set(syns[i]))

In [9]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.114258,-3.921875,0.508789,1.326172,2.685547,1.787109,-1.767578,-1.252930,-1.898438,1.073242,...,0.862305,-0.739746,-1.119141,-0.729004,1.064453,1.273438,-0.246826,1.378906,1.920898,2.220703
1,2.310547,-2.361328,-0.013588,2.707031,1.200195,3.173828,-1.355469,-2.603516,-1.958984,-0.565430,...,0.492920,-0.945801,-0.937500,-0.260986,2.087891,1.500000,-0.222290,0.645020,2.220703,0.959961
2,-0.225098,-2.595703,-0.183105,2.041016,1.673828,1.951172,-1.366211,-2.917969,-0.160645,0.051453,...,-1.433594,-1.019531,-2.537109,-2.455078,2.009766,1.945312,0.622559,1.703125,2.357422,1.230469
3,0.843750,-0.527344,0.271484,0.753906,1.574219,2.496094,-1.651367,-2.703125,-2.267578,1.470703,...,-0.368164,-1.789062,-1.804688,-2.472656,2.876953,0.426025,-0.582031,3.105469,0.837402,1.042969
4,1.519531,-2.650391,0.423584,1.816406,2.031250,1.693359,0.607910,-1.332031,-1.024414,0.936035,...,1.652344,-1.700195,-2.478516,-4.488281,1.403320,0.638672,0.058228,1.249023,2.455078,2.513672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.906738,-1.422852,1.902344,2.380859,1.666992,0.901367,-1.925781,-0.842773,-1.282227,1.339844,...,1.072266,-0.565918,-1.355469,-2.287109,0.418945,1.646484,0.829590,1.502930,2.162109,2.939453
5996,2.136719,-1.451172,0.351074,2.123047,0.430664,-0.171875,-0.983398,-2.193359,-2.029297,1.022461,...,-0.836426,-1.897461,-2.449219,-2.019531,1.227539,0.563477,-1.255859,2.986328,2.496094,1.022461
5997,1.148438,-2.382812,0.937500,0.431152,1.121094,2.527344,-1.701172,-1.219727,-1.711914,1.555664,...,0.711914,-0.082581,-1.974609,-0.666504,2.468750,0.973633,1.819336,-0.183350,2.941406,0.713867
5998,0.944824,-1.447266,-0.097046,0.824707,1.160156,1.099609,-0.375244,-2.130859,-0.812500,2.248047,...,1.913086,0.848145,-1.115234,-1.066406,1.032227,1.030273,-0.781250,0.201294,2.701172,1.934570


In [10]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.114258,-3.921875,0.508789,1.326172,2.685547,1.787109,-1.767578,-1.252930,-1.898438,...,0.862305,-0.739746,-1.119141,-0.729004,1.064453,1.273438,-0.246826,1.378906,1.920898,2.220703
1,5,2.310547,-2.361328,-0.013588,2.707031,1.200195,3.173828,-1.355469,-2.603516,-1.958984,...,0.492920,-0.945801,-0.937500,-0.260986,2.087891,1.500000,-0.222290,0.645020,2.220703,0.959961
2,6,-0.225098,-2.595703,-0.183105,2.041016,1.673828,1.951172,-1.366211,-2.917969,-0.160645,...,-1.433594,-1.019531,-2.537109,-2.455078,2.009766,1.945312,0.622559,1.703125,2.357422,1.230469
3,7,0.843750,-0.527344,0.271484,0.753906,1.574219,2.496094,-1.651367,-2.703125,-2.267578,...,-0.368164,-1.789062,-1.804688,-2.472656,2.876953,0.426025,-0.582031,3.105469,0.837402,1.042969
4,8,1.519531,-2.650391,0.423584,1.816406,2.031250,1.693359,0.607910,-1.332031,-1.024414,...,1.652344,-1.700195,-2.478516,-4.488281,1.403320,0.638672,0.058228,1.249023,2.455078,2.513672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.906738,-1.422852,1.902344,2.380859,1.666992,0.901367,-1.925781,-0.842773,-1.282227,...,1.072266,-0.565918,-1.355469,-2.287109,0.418945,1.646484,0.829590,1.502930,2.162109,2.939453
5996,48466,2.136719,-1.451172,0.351074,2.123047,0.430664,-0.171875,-0.983398,-2.193359,-2.029297,...,-0.836426,-1.897461,-2.449219,-2.019531,1.227539,0.563477,-1.255859,2.986328,2.496094,1.022461
5997,48470,1.148438,-2.382812,0.937500,0.431152,1.121094,2.527344,-1.701172,-1.219727,-1.711914,...,0.711914,-0.082581,-1.974609,-0.666504,2.468750,0.973633,1.819336,-0.183350,2.941406,0.713867
5998,48483,0.944824,-1.447266,-0.097046,0.824707,1.160156,1.099609,-0.375244,-2.130859,-0.812500,...,1.913086,0.848145,-1.115234,-1.066406,1.032227,1.030273,-0.781250,0.201294,2.701172,1.934570


In [11]:
sent_vec_df.to_csv('synopsis_vector.csv', index=False)